# Laser display boards

You're off on your first sightseening trip of your holiday and you need to catch the right llama-rickshaw to get there. You arrive all keen at the llama-rickshaw station, only to find a scene of chaos. The bad news is that there are lots of llama-rickshaws heading to different places. The good news is that above each bay is a display board that shows where that llama-rickshaw is heading. The bad news is that all the display boards have gone down. The good news is that the station staff are handing out the machine-code instructions to generate the messages on the board. 

Given your l33t haxor skilz, it will be no problem to recreate the messages on the display boards.

The board is grid, 80 pixels wide and 8 pixels tall, with row 1 being the top row and column 1 being the left column. The pixels are changed with these commands:

* `top A B` switches the state of the pixels in the topmost row from columns A to B inclusive. If a pixel was lit, it becomes dark; if it was dark, it becomes lit.
* `left A B` is similar, but works on the left edge, toggling the state of pixels in the leftmost column in rows A to B inclusive. 
* `rotate column A B` rotates column A by B spaces down. Pixels that are moved beyond the bottom edge "wrap around" to the top edge.
* `rotate row A B` rotates row A by B spaces to the right. Pixels that are moved beyond the right edge "wrap around" to the left edge.

You can assume all numbers are integers, the row and column values are always valid, and `A` $\le$ `B` in the `left` and `top` commands.

For instance, with a smaller grid that is 10 pixels wide and 4 tall, this is what a sample sequence of instructions would do.

* `top 1 6` turns on the first six pixels on the top row.
```
******....
..........
..........
..........
```

* `rotate column 2 3` moves the lit pixel on the second column to the bottom row.
```
*.****....
..........
..........
.*........
```

* `top 3 10` turns off the pixels in columns 4, 5, and 6, and turns on the pixels in columns 7 to 10.

```
*.....****
..........
..........
.*........
```

* `rotate column 8 5` moves the one lit pixel in column 8 down five rows, wrapping it all the way around the display and leaving the board with one lit pixel in that column one row lower.
```
*.....*.**
.......*..
..........
.*........
```

* `rotate row 2 6` moves that pixel off the right edge of the display, to it wraps around to appear in column 4.
```
*.....*.**
...*......
..........
.*........
```

* `left 1 3` toggles the pixels in rows 1, 2, and 3 of the first column. The top left pixel (previously on) turns off, while the pixels in rows 2 and 3 come on.
```
......*.**
*..*......
*.........
.*........
```

## Part 1
You're standing in front of gate 9¾. You have [the instructions](05-pixels.txt). How many pixels would be lit on the board, if it were working?

# Worked example solution: Parts 1 and 2
This is an example of building an interpreter for a virtual machine, and then executing a program on it. Thinking about the problem in this way allows me to split the problem into two parts straight away:

1. Build a virtual machine
2. Build a parser that will take the string representation of instructions and convert a set of commands the machine will understand. 

The second task is difficult until we've decided what the result of that parsing should look like, so let's look at the virtual machine first.

Neither part is particularly hard. What makes this problem more of a challenge is doing the two parts together, and making sure they mesh in the end.

## Virtual machine

### Representation
The machine is little more than the grid and the operations that act on it. There are only four operations, so let's implement each one as a function that takes as input a machine/grid and returns as output the updated grid (But because of the way Python handles lists, changes are generally done in-place, so we don't _need_ to use the return value.)

We have some choices about how to implement the grid itself. One approach would be a 2d array of booleans, or a 2d array of characters, or of numbers. Python doesn't do 2d arrays (or fixed-sized arrays at all), so arrays would be lists. 

Using a boolean for each cell has the advantage that the `toggle` operations are simple `not` operations. But, if we represent the cells as characters, we can store the grid as a list of strings. To some extent, it doesn't matter with Python, as lists and strings both offer the same interface of picking out individual parts and sections with the slice notation. 

We could also do something like a 'spare array' representation, using a `dict` to store the grid. The keys would be a `tuple` of `(row, column)` and the value would be the boolean/character/number which is the cell at that position. However, as we're taking slices of grid and acting on them, this is likely to get cumbersome. 

For no particular reason, I chose to represent the grid as a list of strings, with a cell being '\*' if that pixel is on and '.' if it's off. This has be slight advantage that printing the grid is easier.

### Commands
You'll notice that the first thing I did was build procedures that create a new grid of a particular size, and print the grid. These are really useful for testing and debugging, as I can easily see what's happening when the other commands go wrong! (The `print_grid` then got a bit more complex to allow for different output formatting.)

The `top` and `left` commands are fairly straightforward, apart from having to be careful with boundary arithmetic for moving between 1-based and 0-based counting, and inclusive-at-both-end sections. 

The `rotate_column` and `rotate_row` functions use the modulus `%` operation to keep the rotation amount down to a sensible level. The `rotate_row` is the simplest: it forms a new row by joining the last few elements of the row to the first few elements of the row, wrapping the last elements to the front. `rotate_column` does the same thing, but with the added complication of snipping out the column into the `col` variable then rebuilding the rows with the `new_col`.

## Parsing
This is much simpler than the first part. 

Each instruction is of the format `<bunch of text> <number> <number>`. That means we can apply the same parsing function to each line without having to worry about which instruction it is, and we can always return the same thing, namely the text and the two numbers. 

Python's `rsplit()` methods splits a string into chunks, by default splitting on whitespace. The `maxsplit` parameter limits how many splits to make, so we don't end up splitting the multi-word command names.

## Applying instructions
One way to do this is with a multi-way `if` statement (or a `case` statement), but that takes space to write and is fragile when it comes to perhaps adding extra commands. Instead, I'm using a _dispatch table_. 

The general idea is that the table contains the functions and procedures we could call, and we pick which one at run time. This simplifies the code in the `interpet` procedure, as all we do is look up the instruction name in the table and apply the function that comes out of it. This is helped by Python's easy syntax for this sort of thing: the name of a function, used without brackets, is the function itself (the brackets mean "apply this function"). 

The `interpret` function just goes along the list of instructions, applying them one at a time. There's some extra bits in there for generating different outputs, and the `clear_output` function is specific to Jupyter notebooks, allowing the next output to be printed in the same place as the last, effectively animating the creation of the message on the display.

In [1]:
import time
import re
from IPython.display import clear_output

In [2]:
WIDTH = 80
HEIGHT = 8

In [3]:
def new_grid(w=WIDTH, h=HEIGHT):
    return ['.' * w for r in range(1, h+1)]

In [4]:
def print_grid(grid, md=False, suppress_dots=False):
    if md:
        print('```')
    for row in grid:
        if suppress_dots:
            print(re.sub(r'\.', ' ', row))
        else:
            print(row)
    if md:
        print('```')

In [5]:
def top(grid, l, r):
    new_segment = ''
    for i in range(l-1, r):
        if grid[0][i] == '.':
            new_segment += '*'
        else:
            new_segment += '.'
    grid[0] = grid[0][:l-1] + new_segment + grid[0][r:]
    return grid

In [6]:
def left(grid, t, b):
    for i in range(t-1, b):
        if grid[i][0] == '.':
            grid[i] = '*' + grid[i][1:]
        else:
            grid[i] = '.' + grid[i][1:]
    return grid

In [7]:
def rotate_column(grid, c, raw_n):
    n = raw_n % len(grid)
    col = [row[c-1] for row in grid]
    new_col = col[-n:] + col[:-n]
    for i in range(len(grid)):
        grid[i] = grid[i][:c-1] + new_col[i] + grid[i][c:]
    return grid

In [8]:
def rotate_row(grid, r, raw_n):
    n = raw_n % len(grid[0])
    grid[r-1] = grid[r-1][-n:] + grid[r-1][:-n]
    return grid

In [9]:
command_dispatch = {'left': left, 'top': top,
                   'rotate row': rotate_row,
                   'rotate column': rotate_column}

In [10]:
def parse(command):
    cmd, a, b = command.rsplit(maxsplit=2)
    return cmd, int(a), int(b)  

In [18]:
def interpret(commands, grid=None, w=WIDTH, h=HEIGHT, 
              show_each_step=False, md=False, overprint=False, overprint_delay=0.25):
    if grid is None:
        grid = new_grid(w, h)
    for c in commands:
        cmd, a, b = parse(c)
        if cmd in command_dispatch:
            grid = command_dispatch[cmd](grid, a, b)
        else:
            raise ValueError('Unknown command')
        if show_each_step:
            if overprint:
                time.sleep(overprint_delay)
            if md: 
                print('`{}`'.format(c))
            else:
                print(c)
            print_grid(grid, md=md)
            print()
            if overprint:
                clear_output(wait=True)
    if show_each_step: 
        print('Final')
        print_grid(grid, md=md)
    return grid

In [12]:
cmds = [c.strip() for c in open('05-pixels.txt').readlines()]
len(cmds)

237

In [13]:
g = interpret(cmds)
sum(1 for c in ''.join(g) if c == '*')

188

In [19]:
g = interpret(cmds, show_each_step=True, overprint=True, overprint_delay=0.1)

Final
...****..............*...................*.....*..............*.................
......*..............*...................***..**..............*.................
......*.*****.*****.****.*****..****.....*.*.***.*****.*****.****.*****..****...
......*.....*.*...*..*.......*..*........*..**.*.....*.*...*..*.......*..*......
......*.*****.*...*..*...*****..*........*..*..*.*****.*...*..*...*****..*......
......*.*...*.*...*..*...*...*..*........*.....*.*...*.*...*..*...*...*..*......
...*..*.*..**.*...*..**..*..**..*........*.....*.*..**.*...*..**..*..**..*......
....**...**.*.*...*...**..**.*..*........*.....*..**.*.*...*...**..**.*..*......


## Part 2
When you've executed your commands, what does the board say your destination is?

In [14]:
print_grid(g)

...****..............*...................*.....*..............*.................
......*..............*...................***..**..............*.................
......*.*****.*****.****.*****..****.....*.*.***.*****.*****.****.*****..****...
......*.....*.*...*..*.......*..*........*..**.*.....*.*...*..*.......*..*......
......*.*****.*...*..*...*****..*........*..*..*.*****.*...*..*...*****..*......
......*.*...*.*...*..*...*...*..*........*.....*.*...*.*...*..*...*...*..*......
...*..*.*..**.*...*..**..*..**..*........*.....*.*..**.*...*..**..*..**..*......
....**...**.*.*...*...**..**.*..*........*.....*..**.*.*...*...**..**.*..*......


In [15]:
print_grid(g, suppress_dots=True)

   ****              *                   *     *              *                 
      *              *                   ***  **              *                 
      * ***** ***** **** *****  ****     * * *** ***** ***** **** *****  ****   
      *     * *   *  *       *  *        *  ** *     * *   *  *       *  *      
      * ***** *   *  *   *****  *        *  *  * ***** *   *  *   *****  *      
      * *   * *   *  *   *   *  *        *     * *   * *   *  *   *   *  *      
   *  * *  ** *   *  **  *  **  *        *     * *  ** *   *  **  *  **  *      
    **   ** * *   *   **  ** *  *        *     *  ** * *   *   **  ** *  *      
